In [1]:
pip install -U langchain-huggingface


Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [5]:
# Load your local document
loader = TextLoader("AI_Full_Course_Overview.txt")
docs = loader.load()

# Split text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


In [6]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(splits, embedding=embeddings)

retriever = vectorstore.as_retriever()


In [13]:
from langchain_community.llms import Ollama
llm = Ollama(model="gemma3")


In [14]:
prompt = ChatPromptTemplate.from_template("""
Use the following context to answer the question:
{context}

Question: {question}
""")


In [15]:
rag_chain = (
    {"context": retriever | (lambda docs: "\n\n".join([d.page_content for d in docs])),
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [19]:
# --- Greeting ---
print("🤖 RAG Chatbot: Hello! How can I help you today?")
print("Type 'exit' or 'quit' to end the chat.\n")

while True:
    # Get user input
    query = input("🧑 You: ")

    # Exit condition
    if query.lower() in ["exit", "quit"]:
        print("🤖 RAG Chatbot: Goodbye! 👋")
        break

    # Get response from RAG chain
    response = rag_chain.invoke(query)

    # Print bot response
    print(f"🤖 RAG Chatbot: {response}\n")


🤖 RAG Chatbot: Hello! How can I help you today?
Type 'exit' or 'quit' to end the chat.



🧑 You:  Explain what topics are covered in the Machine Learning section


🤖 RAG Chatbot: Here’s a breakdown of the topics covered in the Machine Learning section of the AI course, based on the provided overview:

The Machine Learning section covers a broad range of topics, including:

*   **Understanding Learning Types:** It starts with an explanation of supervised, unsupervised, and reinforcement learning.
*   **Key Algorithms:** It delves into several popular machine learning algorithms: Linear Regression, Decision Trees, Random Forest, Support Vector Machines (SVM), and K-Nearest Neighbors (KNN).
*   **Model Training & Evaluation:** The course utilizes scikit-learn for training, testing, and evaluating these models.
*   **Feature Engineering:** A crucial aspect is covered – transforming raw data into suitable input for the models.

Do you want me to elaborate on any specific aspect of the Machine Learning section?



🧑 You:  how many section there


🤖 RAG Chatbot: Based on the provided course overview, there are **6** sections.



🧑 You:  exit


🤖 RAG Chatbot: Goodbye! 👋
